In [8]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

## Mars Exploration Program Scrape

In [4]:
# Import url and parse the html
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [5]:
# latest news title and paragraph text
news_title = soup.find('div', class_='content_title').text
p_text = soup.find('div', class_='image_and_description_container').text

## JPL Scrape

In [6]:
# Import url and parse the html
response = requests.get(url)
soup = bs(response.text, 'html.parser')

In [10]:
# splinter: find the image url for the current Featured Mars Image - `featured_image_url`.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/melissahaynes/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [15]:
# open the url and scrape the featured image
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

# Scrape the featured image
header = soup.find_all('div', class_='header')
for item in header: 
    img = soup.find('a',class_='showimg fancybox-thumbs')['href']


image/featured/mars2.jpg
